### **1、数据clean**

In [1]:
import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
#因为RSTR数据计算的window为500，为了避免删除太多日期，故从08年交易数据开始计算
TRD_Dalyr1 = pd.read_csv("D:/191alphas/return_day_08_22/TRD_Dalyr1.csv")
TRD_Dalyr2 = pd.read_csv("D:/191alphas/return_day_08_22/TRD_Dalyr2.csv")
TRD_Dalyr3 = pd.read_csv("D:/191alphas/return_day_08_22/TRD_Dalyr3.csv")
TRD_Dalyr4 = pd.read_csv("D:/191alphas/return_day_08_22/TRD_Dalyr4.csv")
TRD_Dalyr5 = pd.read_csv("D:/191alphas/return_day_08_22/TRD_Dalyr5.csv")
TRD_Dalyr6 = pd.read_csv("D:/191alphas/return_day_08_22/TRD_Dalyr6.csv")
TRD_Dalyr7 = pd.read_csv("D:/191alphas/return_day_08_22/TRD_Dalyr7.csv")
TRD_Dalyr8 = pd.read_csv("D:/191alphas/return_day_08_22/TRD_Dalyr8.csv")
TRD_Dalyr9 = pd.read_csv("D:/191alphas/return_day_08_22/TRD_Dalyr9.csv")
TRD_Dalyr10 = pd.read_csv("D:/191alphas/return_day_08_22/TRD_Dalyr10.csv")
TRD_Dalyr11 = pd.read_csv("D:/191alphas/return_day_08_22/TRD_Dalyr11.csv")

In [3]:
TRD_Dalyr08_22 = pd.concat([TRD_Dalyr1,TRD_Dalyr2,TRD_Dalyr3,TRD_Dalyr4,TRD_Dalyr5,TRD_Dalyr6,TRD_Dalyr7,TRD_Dalyr8,TRD_Dalyr9,TRD_Dalyr10,TRD_Dalyr11])

In [4]:
TRD_Dalyr08_22.drop(columns = ['Markettype'], inplace = True)

In [5]:
TRD_Dalyr08_22.columns = ['code','date','return_day']

In [6]:
TRD_Dalyr08_22.sort_values(by = ['code','date'], inplace = True)

In [7]:
TRD_Dalyr08_22.reset_index(drop = True, inplace = True)

In [8]:
TRD_Dalyr08_22['date'] = pd.to_datetime(TRD_Dalyr08_22['date'])

In [9]:
TRD_Dalyr08_22

,code,date,return_day
0,1,2008-01-02,-0.016062
1,1,2008-01-03,-0.016588
2,1,2008-01-04,0.027309
3,1,2008-01-07,0.020850
4,1,2008-01-08,0.031146
...,...,...,...
9127532,689009,2022-01-24,0.003281
9127533,689009,2022-01-25,-0.024295
9127534,689009,2022-01-26,-0.010535
9127535,689009,2022-01-27,-0.022423


### **2、半衰加权函数**

In [10]:
#计算半衰期指数权重,时间序列长度为windows，半衰期为half
def Weihgt_half(windows,half):
    list_W = []
    for i in range(windows):
        locals()['W'+str(i)] = 0.5**((windows+1-i)/half)
        list_W.append(locals()['W'+str(i)])
        
    return list_W/pd.Series(list_W).sum()#返回一个数组

In [11]:
#计算半衰加权的函数,column为被加权的列，windows,half为权重计算参数，new_col为加权结果列
def Mean_half(df,column,new_col,windows,half):
    result = pd.DataFrame()
    codelist = df.code.unique()
    Weihgt_h = Weihgt_half(windows, half)
    for code in codelist:
        TRD_sub = df[df.code == code].copy()
        TRD_sub.loc[:,[new_col]] = TRD_sub.rolling(windows)[column].apply(lambda x: (x*Weihgt_h).sum())
        result = pd.concat([result,TRD_sub])
    return result

### **3、计算RSTR**

In [12]:
TRD_Dalyr08_22.loc[:,'ln_return_day'] = np.log(1+TRD_Dalyr08_22['return_day'])

In [13]:
Mean_half_500120 = Mean_half(TRD_Dalyr08_22 ,column = 'ln_return_day', new_col = 'RSTR_0', windows = 500, half = 120)

In [14]:
Mean_half_500120.loc[:,'RSTR'] = Mean_half_500120.groupby('code')['RSTR_0'].shift(20)

In [15]:
Mean_half_500120.to_hdf('C:/Users/Administrator/Downloads/alpha191/RSTR.h5', key = 'RSTR')

In [16]:
Mean_half_500120

,code,date,return_day,ln_return_day,RSTR_0,RSTR
0,1,2008-01-02,-0.016062,-0.016192,NaN,NaN
1,1,2008-01-03,-0.016588,-0.016727,NaN,NaN
2,1,2008-01-04,0.027309,0.026943,NaN,NaN
3,1,2008-01-07,0.020850,0.020636,NaN,NaN
4,1,2008-01-08,0.031146,0.030671,NaN,NaN
...,...,...,...,...,...,...
9127532,689009,2022-01-24,0.003281,0.003276,NaN,NaN
9127533,689009,2022-01-25,-0.024295,-0.024595,NaN,NaN
9127534,689009,2022-01-26,-0.010535,-0.010591,NaN,NaN
9127535,689009,2022-01-27,-0.022423,-0.022678,NaN,NaN


### **4、计算DASTD**

TRD_Dalyr08_22

In [ ]:
#计算日度收益率250日均值
TRD_Dalyr08_22.loc[:,'return_day_mean250'] = TRD_Dalyr08_22.groupby('code',as_index=False)['return_day'].rolling(250).mean()['return_day']

In [ ]:
#计算日度收益率与250日均值的离差平方
TRD_Dalyr08_22.loc[:,'return_d_Sq_dev'] = (TRD_Dalyr08_22['return_day'] - TRD_Dalyr08_22['return_day_mean250'])**2

In [ ]:
#周期250，40天半衰期加权平均，因此需要至少500天交易数据
Mean_half_25040 = Mean_half(TRD_Dalyr08_22 ,column = 'return_d_Sq_dev', new_col = 'DASTD', windows = 250, half = 40)

In [ ]:
Mean_half_25040.isnull().sum()

In [ ]:
Mean_half_25040.info()

In [ ]:
Mean_half_25040_dropna = Mean_half_25040.dropna()

In [ ]:
len(Mean_half_25040_dropna.code.unique())

In [ ]:
Mean_half_25040.to_hdf('C:/Users/Administrator/Downloads/alpha191/DASTD.h5', key = 'DASTD')